In [1]:
import pandas as pd
import numpy as np
import os

In [10]:
test34_umi = pd.read_csv("/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/umi_clean_data/cseq34_CN.umi.csv")
test34_ttl = pd.read_csv("/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/raw_data/cseq34_CN1.no.1.csv")
test34_umi_hamm0 = pd.read_csv("/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/umi_clean_data_hamm0/cseq34_CN.umi.hamm0.csv")
test34_cere1 = pd.read_csv(
    "/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/hamm1_data/cseq34-1.cere.post.BC.hamm1.txt",
    sep="\t", header=None
)
test34_cere2 = pd.read_csv(
    "/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/hamm1_data/cseq34-2.cere.post.BC.hamm1.txt",
    sep="\t", header=None
)
test34_cere1_hamm0 = pd.read_csv(
    "/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/hamm0_data/cseq34-1_cere.post.barcodes.hamm0.txt",
    sep="\t", header=None
)
test34_cere2_hamm0 = pd.read_csv(
    "/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/hamm0_data/cseq34-2_cere.post.barcodes.hamm0.txt",
    sep="\t", header=None
)


In [11]:
len(set(test34_umi["V2"])), len(set(test34_ttl["V2"])), len(set(test34_umi_hamm0["V2"])), test34_cere1[1].nunique(), test34_cere2[1].nunique(), test34_cere1_hamm0[1].nunique(), test34_cere2_hamm0[1].nunique()

(136125, 43895, 43895, 115161, 21776, 23734, 21776)

In [22]:
def umi_collapse(df, sample_id):
    df.columns = ["barcode", "cell", "umi", "reads"]
    df_umiColp = (
        df.groupby(["barcode", "cell"], as_index = False)["umi"]
        .nunique()
    )
    df_umiColp.columns = ["V1", "V2.1", "UMI_Count"]
    df_umiColp["V2"] = sample_id + "_" + df_umiColp["V2.1"].str[5:]
    return df_umiColp[["V1", "V2", "UMI_Count"]]


In [23]:
nuc_hamm1_data_folder = "/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/hamm0_data/"
filenames = os.listdir(nuc_hamm1_data_folder)

In [24]:
filenames

['cseq1_cere.post.barcodes.hamm0.txt',
 'cseq1_pons.pre.barcodes.hamm0.txt',
 'cseq25_cere.post.barcodes.hamm0.txt',
 'cseq25_pons.pre.barcodes.hamm0.txt',
 'cseq31_cere.post.barcodes.hamm0.txt',
 'cseq31_pons.pre.barcodes.hamm0.txt',
 'cseq34-1_cere.post.barcodes.hamm0.txt',
 'cseq34-2_cere.post.barcodes.hamm0.txt',
 'cseq34-1_pons.pre.barcodes.hamm0.txt',
 'cseq34-2_pons.pre.barcodes.hamm0.txt',
 'cseq35_cere.post.barcodes.hamm0.txt',
 'cseq35_pons.pre.barcodes.hamm0.txt',
 'cseq36_cere.post.barcodes.hamm0.txt',
 'cseq36_pons.pre.barcodes.hamm0.txt']

In [25]:
df_dict = {}

for f in filenames:
    sample_name = f.split(".")[0] + "_" + f.split(".")[1]
    sample_df = pd.read_csv(nuc_hamm1_data_folder+f, sep="\t+", engine="python", header=None)
    sample_df.columns = ["barcode", "cell", "umi", "reads"]
    sample_df = sample_df[~sample_df["cell"].str.contains("CY:Z:", na=False)]
    sample_id = f.split(".")[0][4:]
    if sample_id == "34-2": # this step modify the last digit of the CB in 34-2 sample (from "-1" into "-2")
        sample_df["cell"] = sample_df["cell"].str[:-1] + "2"
    if sample_id in ["34-1", "34-2", "36-1", "36-2"]: # this step adds the correct prefix to the CB in cseq34, cseq36 (adding "34" instead of "34-1"/ "34-2", same for 36)
        sample_id = sample_id[0:2]
    df_dict[sample_name] = umi_collapse(sample_df, sample_id)


In [27]:
df_dict.keys()

dict_keys(['cseq1_cere_post', 'cseq1_pons_pre', 'cseq25_cere_post', 'cseq25_pons_pre', 'cseq31_cere_post', 'cseq31_pons_pre', 'cseq34-1_cere_post', 'cseq34-2_cere_post', 'cseq34-1_pons_pre', 'cseq34-2_pons_pre', 'cseq35_cere_post', 'cseq35_pons_pre', 'cseq36_cere_post', 'cseq36_pons_pre'])

In [29]:
df_dict["cseq34_pons_pre"] = pd.concat([df_dict["cseq34-1_pons_pre"], df_dict["cseq34-2_pons_pre"]])
df_dict["cseq34_cere_post"] = pd.concat([df_dict["cseq34-1_cere_post"], df_dict["cseq34-2_cere_post"]])
# df_dict["cseq36_pons"] = pd.concat([df_dict["cseq36-1_pons"], df_dict["cseq36-2_pons"]])
# df_dict["cseq36_cere"] = pd.concat([df_dict["cseq36-1_cere"], df_dict["cseq36-2_cere"]])

# df_dict["cseq36_pons"] = df_dict["cseq36-2_pons"]
# df_dict["cseq36_cere"] = df_dict["cseq36-2_cere"]

In [31]:
out_dir = "/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/umi_clean_data_hamm0/"
file_postfix = {
    "pons": "_PN.umi.hamm0.csv",
    "cere": "_CN.umi.hamm0.csv"
    }

for sample in df_dict.keys():
    if len(sample) <= 16:
        outFile = sample.split("_")[0]+file_postfix[sample.split("_")[1]]
        df_dict[sample].to_csv(out_dir+outFile, index=False)

In [ ]:
### The following section is to check whether the final connectome barcode is presented in the umiCleaned neuron.

In [3]:
connectome_match = pd.read_csv("/home/zhouwan2/expansion_link/Cseq1_data/connectome_data/connectome.match.count.csv")
nuc_umi_clean = "/home/zhouwan2/expansion_link/Cseq1_data/nuc_data/nuc_umi_clean_data/"

nuc_umi_clean_file = os.listdir(nuc_umi_clean)
nuc_umi_clean_file

['cseq36_PN.neuron.umiClean.csv',
 'cseq36_CN.neuron.umiClean.csv',
 'cseq35_PN.neuron.umiClean.csv',
 'cseq35_CN.neuron.umiClean.csv',
 'cseq34_PN.neuron.umiClean.csv',
 'cseq34_CN.neuron.umiClean.csv',
 'cseq31_CN.neuron.umiClean.csv',
 'cseq31_PN.neuron.umiClean.csv',
 'cseq25_PN.neuron.umiClean.csv',
 'cseq25_CN.neuron.umiClean.csv',
 'cseq1_PN.neuron.umiClean.csv',
 'cseq1_CN.neuron.umiClean.csv']

In [6]:
pre_nuc, post_nuc = {}, {}
for i in nuc_umi_clean_file:
    region = i.split(".")[0][-2:]
    sample_id = i.split(".")[0].split("_")[0][4:]
    df = pd.read_csv(nuc_umi_clean+i)
    if region == "PN":
        pre_nuc[sample_id] = df
    else:
        post_nuc[sample_id] = df



1_TGTCCCAAGGAAAGAC-1
1_TGGTACACATAGGTAA-1
1_AACGTCACAACGACAG-1
1_ATACTTCGTGTACATC-1
1_ATTCATCTCAAACCTG-1
1_TCGTGGGCAGCGACCT-1
1_ACACAGTGTACCCGCA-1
1_ACATTTCCAGCACGAA-1
1_AGGGCCTAGTCACTAC-1
1_ATTATCCGTTGCGAAG-1
1_CGGGACTCAGACCAGA-1
1_ATCATTCGTGACTGAG-1
1_ATCGTCCGTACGCGTC-1
1_TAGCACAGTTTACCAG-1
1_TCCGTGTCATCATCTT-1
1_ATGGTTGCAAGGCGTA-1
1_CTATCTAGTATTAAGG-1
1_CTCCTCCGTATCGCTA-1
1_ATCGTCCGTACGCGTC-1
1_GACTTCCAGTTATGGA-1
1_GGGCTACGTCCCAAAT-1
1_CGAGGAATCTAGCCAA-1
1_GTCAAGTAGCGTTACT-1
1_GTGGAAGCATCAGCTA-1
1_TACAACGCAGCGGTTC-1
1_GGGCGTTAGGATCATA-1
1_TAGCACAAGACATAGT-1
1_TGCAGGCCAGCCATTA-1
1_ACCAAACAGCTTTCTT-1
1_CCATCACCACCGGAAA-1
1_GCCAGGTTCGTACCTC-1
1_TGTGCGGAGATCGCCC-1
1_ATACCTTAGGTGGCTA-1
1_CCATCACCACCGGAAA-1
25_AAACCCAAGGACTAAT-1
25_AAACGCTTCCGTGGGT-1
25_AAAGAACAGTCGGCAA-1
25_TGCTGAAAGTTCACTG-1
25_TACTTGTCATAATCCG-1
25_GAGAAATGTGAGCTCC-1
25_AAGATAGAGAAGGGAT-1
25_AATGCCATCTCTCCGA-1
25_TTCTTCCGTCGTTATG-1
25_ACACGCGCACACCAGC-1
25_ACCATTTAGAAGCCTG-1
25_GCCAGCAAGTAATACG-1
25_ACGCACGTCCAACCGG-1


In [9]:
for i in connectome_match["preNuc_CB"]:
    sample_id = i.split("_")[0]
    corres_df_col = pre_nuc[sample_id]["V2"]
    if i not in corres_df_col.values:
        print(i)

In [10]:
for i in connectome_match["postNuc_CB"]:
    sample_id = i.split("_")[0]
    corres_df_col = post_nuc[sample_id]["V2"]
    if i not in corres_df_col.values:
        print(i)

34_GTGTGGCTCTCCTGCA-1
34_TGTCAGATCCATGAGT-1
34_CAAGAGGAGAAAGTCT-1
34_CTACCCAGTTAACAGA-1
34_TGTCAGATCCATGAGT-1
34_TCCCACAAGGGTAATT-1
34_ACAAGCTTCGGCTTGG-1
34_CTGCGAGGTAGGATAT-1
34_TGTCAGATCCATGAGT-1
34_CAAGAGGAGAAAGTCT-1
34_CTACCCAGTTAACAGA-1
34_CTACCCAGTTAACAGA-1
34_CTACCCAGTTAACAGA-1
34_TGTCAGATCCATGAGT-1
